## Differential Equations: Computational practicum
Author: Vyacheslav Shults, BS17-8

In [1]:
# setup the plotly graphics library and configure it to show 
# figures inline in the notebook
import plotly.plotly as py
import plotly.graph_objs as go

import numpy as np
from math import e

In [2]:
# make qutip available in the rest of the notebook
# from qutip import *
import warnings
warnings.filterwarnings('ignore')

# Introduction

The goal of this practicum is to solve Initial value problem (IVP) with an interesting interval using numerical methods such that Euler's method, improved Euler’s method and Runge-Kutta method.
### $$
\left\{\begin{matrix}
  y'=f(x, y)\\ 
  y(x_{0})=y_{0}\\
  x\in [x_{0}, X]
\end{matrix}\right.
$$
In Variant 14 we have $ f(x, y) = \cos\left(x\right)-y$,  $x_{0}=1, \,y_{0}=1,\, X=9.5$

## Exact solution
#### $ y' = \cos\left(x\right)-y$
This equation has exact solution $y=ce^{-x}+\frac{1}{2}\sin(x)+\frac{1}{2}\cos(x)$

According to the initial value constant $c=e^{x_{0}}\left(y_{0}-\frac{\sin(x_{0})+\cos(x_{0})}{2}\right)$ 

Solution is $y=\frac{1}{2}\left(2y_{0}-\sin(x_{0})-\cos(x_{0})\right)e^{x_{0}-x}+\frac{1}{2}\sin(x)+\frac{1}{2}\cos(x)$

In [3]:
# Define function f(x, y)
f = lambda x, y: np.cos(x)-y
# Define x0, y0, X
x0 = 1
y0 = 1
X = 9.5

# Define number of steps
n_steps = 1000

# Define exact solution
f_e = lambda x, x0=x0, y0=y0: (y0-(np.sin(x0)+np.cos(x0))/2)*e**(x0-x)+(np.sin(x)+np.cos(x))/2

## Euler's method

In [4]:
def euler(fun, x0, y0, X, n_steps, print_res=False):
    x = x0
    y = y0
    
    h = (X - x)/n_steps
    res = []
    
    for i in np.arange(n_steps):
        res.append((i, x, y))
        y += fun(x, y)*h
        x += h
        if print_res: 
            print("n: {:<4} x: {:<12g} y: {:<12g}".format(*res[-1]))
    return np.array(res)

## Improved Euler's method

In [5]:
def imroved_euler(fun, x0, y0, X, n_steps, print_res=False):
    x = x0
    y = y0
    
    h = (X - x)/n_steps
    res = []
    
    for i in np.arange(n_steps):
        res.append((i, x, y))
        y_h = y + fun(x, y)*h
        y = y + (fun(x, y)+fun(x+h, y_h))*h/2
        x = x + h
        if print_res: 
            print("n: {:<4} x: {:<12g} y: {:<12g}".format(*res[-1]))
    return np.array(res)

## Runge–Kutta method

In [6]:
def runge_kutta(fun, x0, y0, X, n_steps, print_res=False):
    x = x0
    y = y0
    
    h = (X - x)/n_steps
    res = []
    
    for i in np.arange(n_steps):
        res.append((i, x, y))
        k1 = h*fun(x, y)
        k2 = h*fun(x + h/2, y + k1/2)
        k3 = h*fun(x + h/2, y + k2/2)
        k4 = h*fun(x + h, y + k3)
        y = y + (k1 + 2*k2 + 2*k3+ k4)/6
        x = x + h
        if print_res: 
            print("n: {:<4} x: {:<12g} y: {:<12g}".format(*res[-1]))
    return np.array(res)

In [7]:
res1 =         euler(f, x0, y0, X, n_steps)
res2 = imroved_euler(f, x0, y0, X, n_steps)
res3 =   runge_kutta(f, x0, y0, X, n_steps)

In [8]:
trace1 = go.Scatter(
    name = "Exact solution",
    x = res1[:,1],
    y = f_e(res1[:,1])
)
trace2 = go.Scatter(
    name = "Euler method",
    x=res1[:,1],
    y=res1[:,2],
    xaxis='x',
    yaxis='y',  
)
trace3 = go.Scatter(
    name = "Improved Euler method",
    x=res2[:,1],
    y=res2[:,2],
    xaxis='x',
    yaxis='y',  
)
trace4 = go.Scatter(
    name = "Runge–Kutta method",
    x=res3[:,1],
    y=res3[:,2],
    xaxis='x',
    yaxis='y',  
)
layout = go.Layout(title="Solutions of ODE",
                   xaxis=dict(title="X axis"),
                   yaxis=dict(title="Y axis"))
data = [trace1, trace2, trace3, trace4]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='simple-inset')

In [9]:
tr_err1 = go.Scatter(
    name = "Euler method",
    x = res1[:,1],
    y = np.log10(np.abs(res1[:,2]-f_e(res1[:,1]))),
    xaxis = 'x',
    yaxis = 'y',  
)
tr_err2 = go.Scatter(
    name = "Improved Euler method",
    x = res2[:,1],
    y = np.log10(np.abs(res2[:,2]-f_e(res1[:,1]))),
    xaxis = 'x',
    yaxis = 'y',  
)
tr_err3 = go.Scatter(
    name = "Runge–Kutta method",
    x = res3[:,1],
    y = np.log10(np.abs(res3[:,2]-f_e(res3[:,1]))),
    xaxis = 'x',
    yaxis = 'y',  
)
layout_ltr = go.Layout(title="Comparison of Logarithm of global truncation errors",
                       xaxis=dict(title="X axis"),
                       yaxis=dict(title="Log10 error"))
data = [tr_err1, tr_err2, tr_err3]
fig = go.Figure(data=data, layout=layout_ltr)
py.iplot(fig, filename='comparison_of_gtr')

## Bothside Euler's method

In [10]:
def bs_euler(fun, x0, y0, X_start, X_end, n_steps):
    if not (X_start <= x0 <= X_end):
        return np.array([])
    x = x0
    y = y0
    
    h = (X_end - X_start)/n_steps
    res = []
    left_part = np.arange(x0, X_start, -h)
    right_part = np.arange(x0, X_end, h) 
            
    for x in left_part:
        res.append((x, y))
        y -= fun(x, y)*h
        
    res.reverse()
    y = y0
            
    for x in right_part:
        res.append((x, y))
        y += fun(x, y)*h
    
    
    return np.array(res)

## Bothside Improved Euler's method

In [11]:
def bs_imroved_euler(fun, x0, y0, X_start, X_end, n_steps):
    if not (X_start <= x0 <= X_end):
        return np.array([])
    x = x0
    y = y0
    
    h = (X_end - X_start)/n_steps
    res = []
    left_part = np.arange(x0, X_start, -h)
    right_part = np.arange(x0, X_end, h)
    
    for x in left_part:
        res.append((x, y))
        y_h = y - fun(x, y)*h
        y = y - (fun(x, y)+fun(x-h, y_h))*h/2
        
    res.reverse()
    y = y0
            
    for x in right_part:
        res.append((x, y))
        y_h = y + fun(x, y)*h
        y = y + (fun(x, y)+fun(x+h, y_h))*h/2
#         x = x + h
    
    return np.array(res)

## Bothside Runge–Kutta method

In [12]:
def bs_runge_kutta(fun, x0, y0, X_start, X_end, n_steps, print_res=False):
    if not (X_start <= x0 <= X_end):
        return np.array([])
    x = x0
    y = y0
    
    h = (X_end - X_start)/n_steps
    res = []
    left_part = np.arange(x0, X_start, -h)
    right_part = np.arange(x0, X_end, h)
    
    for x in left_part:
        res.append((x, y))
        k1 = -h*fun(x, y)
        k2 = -h*fun(x - h/2, y + k1/2)
        k3 = -h*fun(x - h/2, y + k2/2)
        k4 = -h*fun(x - h, y + k3)
        y = y + (k1 + 2*k2 + 2*k3 + k4)/6  
        
    res.reverse()
    y = y0
    
    for x in right_part:
        res.append((x, y))
        k1 = h*fun(x, y)
        k2 = h*fun(x + h/2, y + k1/2)
        k3 = h*fun(x + h/2, y + k2/2)
        k4 = h*fun(x + h, y + k3)
        y = y + (k1 + 2*k2 + 2*k3+ k4)/6  
    
    return np.array(res)

In [13]:
x_start = -5
x_end = 20
bs_eu = bs_euler(f, x0, y0, x_start, x_end, n_steps*2)
bs_ie = bs_imroved_euler(f, x0, y0, x_start, x_end, n_steps*2)
bs_rk = bs_runge_kutta(f, x0, y0, x_start, x_end, n_steps*2)

In [14]:
trace1_bs = go.Scatter(
    name = "Exact solution",
    x = bs_eu[:,0],
    y = f_e(bs_eu[:,0])
)
trace2_bs = go.Scatter(
    name = "Bothside Euler method",
    x=bs_eu[:,0],
    y=bs_eu[:,1],
    xaxis='x',
    yaxis='y',  
)
trace3_bs = go.Scatter(
    name = "Bothside Improved Euler method",
    x=bs_ie[:,0],
    y=bs_ie[:,1],
    xaxis='x',
    yaxis='y',  
)
trace4_bs = go.Scatter(
    name = "Bothside Runge–Kutta method",
    x=bs_rk[:,0],
    y=bs_rk[:,1],
    xaxis='x',
    yaxis='y',  
)
layout = go.Layout(title="Solutions of ODE",
                   xaxis=dict(title="X axis"),
                   yaxis=dict(title="Y axis"))
data = [trace1_bs, trace2_bs, trace3_bs, trace4_bs]
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='simple-inset')

In [15]:
tr_err1_bs = go.Scatter(
    name = "Bothside Euler method",
    x = bs_eu[:,0],
    y = np.log10(np.abs(bs_eu[:,1]-f_e(bs_eu[:,0]))),
    xaxis = 'x',
    yaxis = 'y',  
)
tr_err2_bs = go.Scatter(
    name = "Bothside Improved Euler method",
    x = bs_ie[:,0],
    y = np.log10(np.abs(bs_ie[:,1]-f_e(bs_ie[:,0]))),
    xaxis = 'x',
    yaxis = 'y',  
)
tr_err3_bs = go.Scatter(
    name = "Bothside Runge–Kutta method",
    x = bs_rk[:,0],
    y = np.log10(np.abs(bs_rk[:,1]-f_e(bs_rk[:,0]))),
    xaxis = 'x',
    yaxis = 'y',  
)
layout_gtr_bs = go.Layout(title="Comparison of Logarithm of global truncation errors",
                       xaxis=dict(title="X axis"),
                       yaxis=dict(title="Log10 error"))
data = [tr_err1_bs, tr_err2_bs, tr_err3_bs]
fig = go.Figure(data=data, layout=layout_ltr)
py.iplot(fig, filename='comparison_of_gtr')